In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split

In [3]:
# Load Data

data = pd.read_csv("data/weatherHistory.csv")
dates = pd.DataFrame(data.iloc[:,0:1].values)
temp_data = data.iloc[:,3:4].values #temperature column only
temp_data
print(type(dates))

<class 'pandas.core.frame.DataFrame'>


In [4]:
# Optional Filter

filter_active = 0
if filter_active == True:
  temp_data = medfilt(temp_data, 3)
  temp_data = gaussian_filter1d(temp_data, 1.2)

In [5]:
# Encode the data

mms = MinMaxScaler(feature_range = (0,1))
temp_data = mms.fit_transform(temp_data)

In [6]:
# Train data (OPTION 1)

x_train = []
y_train = []
n_future = 7     #7 days temperature forecast
n_past = 30      #Past 30 days 

for i in range(0,len(temp_data)-n_past-n_future+1):
    x_train.append(temp_data[i : i + n_past , 0])     
    y_train.append(temp_data[i + n_past : i + n_past + n_future , 0 ])

x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )


# ----------------------------------------------------------------------------

# Train Data (OPTION 2)

# x_train, y_train = train_test_split(temp_data, 
#                                     test_size=0.25, 
#                                     random_state=0)

In [7]:
# LSTM Model Defninition

model = Sequential()
model.add(Bidirectional(LSTM(units=50, 
                             activation= "relu", 
                             return_sequences=True, 
                             input_shape = (x_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(LSTM(units= 50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 50))
model.add(Dense(units = n_future , activation="relu"))  

In [8]:
# Extra Model Functionality

# Model Checkpoint 
checkpoint_filepath = '/checkpoints'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='max',
    save_best_only=True)


# Early Stopping
stopping = EarlyStopping(monitor='loss', patience=5)


In [10]:

# Train Model

model.compile(optimizer='adam', 
              loss = 'mean_squared_error', 
              metrics='accuracy')
history = model.fit(x_train, 
          y_train, 
          epochs=10, 
          batch_size=128,
          verbose=1,
          callbacks=[checkpoint, stopping]
          )

# Save model
model.save('lstm_model.h5')

Epoch 1/10
754/754 [==============================] - ETA: 0s - loss: 0.0529 - accuracy: 0.2347

PermissionDeniedError: /checkpoints_temp_b8b626421f7f4330b0b3ebdd1206f3ff; Read-only file system [Op:SaveV2]

In [ ]:

# Visualize Metrics

# Loss
def visualize_loss(history, title):
    loss = history.history["loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "ro--", label="Training loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

# Accuracy
def visualize_acc(history, title):
    acc = history.history["accuracy"]
    epochs = range(len(acc))
    plt.figure()
    plt.plot(epochs, acc, "ro--", label="Training loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()



visualize_loss(history, "Training Loss")
visualize_acc(history, "Training Accuracy")


In [ ]:
# Predictions and Plots
temp_data


array([[0.50697507],
       [0.50508505],
       [0.50544505],
       ...,
       [0.71055711],
       [0.70218702],
       [0.68463685]])

In [ ]:
# Prediction

